In [20]:
import pandas as pd
import numpy as nb
import matplotlib.pyplot as plt
import seaborn as sns
import os
import openai
import nltk

plt.style.use('ggplot')

openai.api_key = os.getenv("OPENAI_API_KEY")

In [12]:
file_path = 'interactions_table_nov_jan.csv'
df = pd.read_csv(file_path)

In [13]:
#select 50 interactions where the user is different from "NOC", "Chat GPT" or "Prompt" and those that does not contains the workd "teste"
df2 = df.loc[~df['user'].isin(["NOC", "Chat GPT", "Prompt", "Felipe undefined"]) 
                & ~df['user'].str.contains("teste") 
                & ~df['message'].str.contains("Não entendi, escolha uma das opções") 
                & ~df['message'].str.contains("Opção Inválida") 
                & ~df['message'].str.contains("Escolha uma das seguintes opções")
                & ~df['message'].str.contains("não entendemos sua mensagem")
                & ~df['message'].str.contains("Departamento Fiscal/Contábil")
                & ~df['message'].str.contains("agradece seu contato")
                & ~df['message'].str.contains("agradece o seu contato")
                & ~df['message'].str.contains("agradece contato")
                & ~df['message'].str.contains("😃")
                & ~df['message'].str.contains("Seja bem-vindo")]

In [14]:
df3 = df2

In [15]:
#select only the chat_id group that has more than 20 interactions
df4 = df3.groupby('chat_id').filter(lambda x: len(x) > 10)

In [16]:
#una todos os textos da coluna "message" pelo chat_id em uma única string, e crie uma nova df com essas strings, o chat_id, o user e a data (created_at) da última mensagem

df5 = df4.groupby('chat_id').agg({'message': ' '.join, 'user': 'last', 'created_at': 'last'}).reset_index()

In [52]:
def translate_message(text):
    prompt = f"""Você é um tradutor que analisa mensagens e traduz do português para o inglês
     Se não tiver certeza de uma tradução, você pode dizer "Não tenho certeza" e recomendar a revisão do usuário manualmente.

     Analise o texto a seguir e faça a tradução do português para o inglês.
    {text}"""

    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "Você é um tradutor prestativo."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=60,
            temperature=0
        )

        translation = response.choices[0].message['content'].strip().lower()
        return translation
    except Exception as e:
        print("Ocorreu um erro:", e)
        return "Erro"

In [18]:
example = df5['message'][10]
print(example)

A minha estar cobrando Taxa de aluguel da maquina E no cnpj Queria que me enviasse a máquina E queria a máquina de cartai Então já tenho a conta blu Mais uso o aplicativo da blu E ela foi coletada Eu usava a da getnet Queria que me enviasse uma máquina blu Olá, quero falar com o suporte da Blu E queria fazer o pedido Mais não tenho a máquina Já uso o portal Qual o site de fazer o pedido da máquina Queria fazer o pedido de uma máquina de cartão Eunice


In [30]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax  

In [31]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

c:\Users\Larissa Lorenzi\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Larissa Lorenzi\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


c:\Users\Larissa Lorenzi\AppData\Local\Programs\Python\Python311\Lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [32]:
example2 = 'O produto não é bom, não gostei, cheio de defeitos'
print(example2)

O produto não é bom, não gostei, cheio de defeitos


In [36]:
encoded_text = tokenizer(example2, return_tensors="pt")
output = model(**encoded_text)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
scores_dict = {
    'roberta_neg': scores[0],
    'roberta_neu': scores[1],
    'roberta_pos': scores[2]
}

print(scores_dict)

{'roberta_neg': 0.20471664, 'roberta_neu': 0.7116031, 'roberta_pos': 0.08368025}


In [38]:
def polarity_scores_roberta(example):
    encoded_text = tokenizer(example, return_tensors="pt")
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'roberta_neg': scores[0],
        'roberta_neu': scores[1],
        'roberta_pos': scores[2]
    }
    return scores_dict

In [55]:
res = {}
for i, row in tqdm(df5.iterrows(), total=len(df5)):
    try:    
        text = row['message_trans']
        myid = row['chat_id']
        roberta_results = polarity_scores_roberta(text)
        res[myid] = roberta_results
    except RuntimeError:
        print(f'Broke for chat_id {chat_id}')

  0%|          | 0/30 [00:00<?, ?it/s]

In [56]:
results_df = pd.DataFrame(res).T
results_df = results_df.reset_index().rename(columns={'index': 'chat_id'})
results_df = results_df.merge(df5, how='left')

In [57]:
results_df.head()

,chat_id,roberta_neg,roberta_neu,roberta_pos,message,user,created_at,message_trans
0,14VVFZN7R80UNPVA8Q04,0.129920,0.732714,0.137366,Empresa Preciso cadastrsr outra empresan __aud...,Seoneco 🐸,2023-11-27T13:20:31.515216-03:00,company i need to register another company __a...
1,1DP3MDANB0OGQKASW1OM,0.119383,0.481679,0.398939,O cadastro já deu certo agora só falta pedir a...,Welder Vendedor,2024-01-02T12:35:00.727364-03:00,"the registration has already been successful, ..."
2,1OHYT505S3V28ELHD28U,0.080613,0.812238,0.107149,Mais eu sou uma criança Como faço Eu sou uma c...,Tulio undefined,2023-11-30T11:02:34.54443-03:00,"desculpe, mas o texto fornecido é muito confus..."
3,25472dae-41be-4db5-b465-92b3ee0f285d,0.124147,0.808101,0.067752,"Meu salário é 2000,00 e vou tirar 10 dias de f...",Thaissa Delgado,2023-12-04T19:42:38.730139-03:00,my salary is $2000.00 and i'm going to take 10...
4,2S8DYFBPKE8253H7EFPH,0.007801,0.143044,0.849155,O pagamento em 12x para o cliente tem juros? E...,Cidadedossonhosmoveis undefined,2023-12-07T14:08:58.011084-03:00,does the 12x payment for the customer have int...


In [53]:
df5['message_trans'] = df5['message'].apply(translate_message)

In [58]:
#mostre tudo sobre o chat_id 3HX5TD1K48QBH4IDESXZ

results_df[results_df['chat_id'] == '3HX5TD1K48QBH4IDESXZ']

,chat_id,roberta_neg,roberta_neu,roberta_pos,message,user,created_at,message_trans
5,3HX5TD1K48QBH4IDESXZ,0.869413,0.116679,0.013909,"pode deixar, sua ajuda muito é de pouca avalia...",Fernando Bronté,2023-12-13T11:02:26.117422-03:00,"you can leave it, your help is of little value..."


In [50]:
df5.loc[df5['chat_id'] == '3HX5TD1K48QBH4IDESXZ', 'message'].iloc[0]

'pode deixar, sua ajuda muito é de pouca avalia pode deixar, sua ajuda muito é de pouca avalia Sistema para da a informação que já existe um usuario com um dos dados informados já informei várias vezes o que esta acontecendo mas nao ´parece Péssimo serviço Repete sempre a mesma coisa Então ajuda da a informação que já existe um usuario com um dos dados informados da a informação que já existe um usuario com um dos dados informados já tentei mas não avança preciso cadastrar outro email e usuario da conta blu'

In [51]:
results_df.columns

Index(['chat_id', 'roberta_neg', 'roberta_neu', 'roberta_pos', 'message',
       'user', 'created_at'],
      dtype='object')

In [ ]:
sns.scatterplot(data=results_df, x='roberta_neg', y='roberta_pos', z='roberta_neu', hue='user')